Following: https://github.com/strongio/keras-bert/blob/master/keras-bert.ipynb

In [4]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
import numpy as np
import pandas as pd

In [6]:
from sklearn.metrics import classification_report

In [7]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import backend as K

In [8]:
# Initialize session
sess = tf.Session()

#### Custom Functions

In [9]:
from src.callbacks import PlotCurves
from src.custom_functions import f1_score
from src.load_data import load_data

Using TensorFlow backend.


### Load Data

In [10]:
train_data, valid_data, test_data, metadata = load_data()

In [11]:
max([len(sentence['sentence']) 
     for article in (train_data + valid_data + test_data)
     for sentence in article['sentences']])

639

### Prepare data

In [12]:
n_tags = 2
label_list = [0, 1]
max_seq_length = 640

In [13]:
X_tra = np.array([sentence['sentence'].replace('\n', '').strip() 
                  for article in train_data 
                  for sentence in article['sentences']])
X_val = np.array([sentence['sentence'].replace('\n', '').strip() 
                  for article in valid_data 
                  for sentence in article['sentences']])
X_test = np.array([sentence['sentence'].replace('\n', '').strip() 
                   for article in test_data 
                   for sentence in article['sentences']])

In [14]:
y_tra = np.array([sentence['label'] for article in train_data for sentence in article['sentences']])
y_val = np.array([sentence['label'] for article in valid_data for sentence in article['sentences']])
y_test = np.array([sentence['label'] for article in test_data for sentence in article['sentences']])

### Prepare Data

In [17]:
from src.keras_bert import convert_text_to_examples, \
                           create_tokenizer_from_hub_module, \
                           convert_examples_to_features, \
                           BertLayer

In [18]:
bert_path = "https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1"

In [21]:
tokenizer = create_tokenizer_from_hub_module(bert_path)

In [22]:
train_examples = convert_text_to_examples(X_tra, y_tra)
valid_examples = convert_text_to_examples(X_val, y_val)

In [23]:
train_input_ids, train_input_masks, train_segment_ids, train_labels = \
            convert_examples_to_features(tokenizer, train_examples, max_seq_length)

Converting examples to features: 100%|██████████| 3582/3582 [00:02<00:00, 1449.13it/s]


In [24]:
valid_input_ids, valid_input_masks, valid_segment_ids, valid_labels = \
            convert_examples_to_features(tokenizer, valid_examples, max_seq_length)

Converting examples to features: 100%|██████████| 399/399 [00:00<00:00, 1416.78it/s]


### BERT

In [27]:
# Build model
def build_model(max_seq_length):
    
    in_id = tf.keras.layers.Input(shape=(max_seq_length,), name="input_ids")
    in_mask = tf.keras.layers.Input(shape=(max_seq_length,), name="input_masks")
    in_segment = tf.keras.layers.Input(shape=(max_seq_length,), name="segment_ids")
    bert_inputs = [in_id, in_mask, in_segment]
    
    bert_output = BertLayer(n_fine_tune_layers=2, pooling="first", bert_path=bert_path)(bert_inputs)
    
    dense = tf.keras.layers.Dense(128, activation='relu')(bert_output)
    
    pred = tf.keras.layers.Dense(1, activation='sigmoid')(dense)
    
    return tf.keras.models.Model(inputs=bert_inputs, outputs=pred)

In [29]:
model = build_model(max_seq_length)
model.summary()

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_ids (InputLayer)          [(None, 640)]        0                                            
__________________________________________________________________________________________________
input_masks (InputLayer)        [(None, 640)]        0                                            
__________________________________________________________________________________________________
segment_ids (InputLayer)        [(None, 640)]        0                                            
__________________________________________________________________________________________________
bert_layer_1 (BertLayer)        (None, 768)          110104890   input_ids[0][0]                  
                                                                 input_masks[0][0]          

In [30]:
sess.run(tf.local_variables_initializer())
sess.run(tf.global_variables_initializer())
sess.run(tf.tables_initializer())
K.set_session(sess)

In [31]:
model.fit(
    [train_input_ids, train_input_masks, train_segment_ids], 
    train_labels,
    validation_data=([valid_input_ids, valid_input_masks, valid_segment_ids], valid_labels),
    epochs=1,
    batch_size=32
)

Train on 3582 samples, validate on 399 samples


InvalidArgumentError: 2 root error(s) found.
  (0) Invalid argument: assertion failed: [] [Condition x <= y did not hold element-wise:x (bert/embeddings/strided_slice_3:0) = ] [640] [y (bert/embeddings/assert_less_equal/y:0) = ] [512]
	 [[{{node bert_layer_1/bert_layer_1_module_apply_tokens/bert/embeddings/assert_less_equal/Assert/Assert}}]]
	 [[loss_1/mul/_1935]]
  (1) Invalid argument: assertion failed: [] [Condition x <= y did not hold element-wise:x (bert/embeddings/strided_slice_3:0) = ] [640] [y (bert/embeddings/assert_less_equal/y:0) = ] [512]
	 [[{{node bert_layer_1/bert_layer_1_module_apply_tokens/bert/embeddings/assert_less_equal/Assert/Assert}}]]
0 successful operations.
0 derived errors ignored.